In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from IPython.display import HTML, display


Data source/credit: https://www.kaggle.com/aungpyaeap/supermarket-sales


**Supermarket Analysis**

* Branch: understand branch performance and drivers of growth 
* Product: which products should the stores continue to sell and prioritize, identify top sellers and best gross margin 
* Customer: what is the breakdown of customers (Member v Normal, Male v. Female)
* Payments: how are customers paying? Does payment type affect sales revenue?******

In [ ]:
#Load the data
data = pd.read_csv('../input/supermarket-sales/supermarket_sales - Sheet1.csv')

In [ ]:
#Preview
data.head()

In [ ]:
data.info()

Numerical Variables: Unit Price,Quantity, Tax, COGs, Gross Margin %, Gross Income, Rating

Categorical Variables: Branch, City, Customer Type, Gender, Product Line, Payment

In [ ]:
categorical = data[["Branch","City","Customer type","Gender","Product line","Payment"]]
categorical.nunique()


In [ ]:
def get_unique_count(var):
    return data[var].value_counts()

for var in categorical:
     results = pd.DataFrame(get_unique_count(var))
     print(results)

In [ ]:
data.describe()

In [ ]:
data.shape

In [ ]:
#check for missing data
missing = data.isnull().sum()
missingcheck = missing[missing > 0]
missingcheck

#There are no missing data entries

**Branch Performance Analysis**

In [ ]:
plt.figure(figsize= (9,6))


g1 = sns.distplot(data[data['Branch']=='A']['gross income'], hist=True, label="A")
g1 = sns.distplot(data[data['Branch']=='B']['gross income'], hist=True, label="B")
g1 = sns.distplot(data[data['Branch']=='C']['gross income'], hist=True, label="C")


In [ ]:
data['gross income'].describe()

In [ ]:
#Gross Income Distrbution
print(data['gross income'].skew())
print(data['gross income'].kurt())

In [ ]:
plt.figure(figsize = (8,5))

p = sns.boxplot(x=data['Branch'], y=data['gross income'], data=data, palette="Set1")
p.set_xticklabels(p.get_xticklabels())
p.set_title("Gross Income by Branch", fontsize=15)
p.set_xlabel("",fontsize=15)
p.set_ylabel("Gross Income", fontsize=15)

plt.show()

In [ ]:
print(data['gross income'].quantile(0.99))
data[data['gross income'] > 45.25]

In [ ]:
quantity_by_branch = data.groupby("Branch")["Quantity","gross income"].sum()
quantity_by_branch

We need to transform the date data since it was an object type

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data['Time'] = pd.to_datetime(data['Time'])
#no need to include year since there is only one year (2019)
data['day'] = (data['Date']).dt.day
data['month'] = (data['Date']).dt.month
data['hour'] = (data['Time']).dt.hour
data['weekday'] = (data['Date']).dt.day_name()

Our sales data spans from Jan 1, 2019 to March 30, 2019. Has the quantity sold been increasing over the past few months? 

In [ ]:
import matplotlib.dates as mdates

sorted = data.sort_values(by='Date')
sorted = sorted.groupby('Date')['Quantity','gross income'].sum().reset_index()
sorted

months = mdates.MonthLocator() 

plt.figure(figsize=(12,6))
g1 = sns.pointplot(sorted['Date'], sorted['Quantity'], label='Quantity')
g1.xaxis.set_major_locator(months)

plt.show()

There isn't an overall increase in quantity sold over the three months, however there are peaks in February and March

**Product Analysis**

In [ ]:
product_avg = data.groupby("Product line")["gross income","Unit price","Quantity"].mean().reset_index()
product_avg

In [ ]:
product_totals = data.groupby("Product line")["Quantity"].sum()


In [ ]:
sns.barplot(x=product_totals.index, y=product_totals.values, alpha=0.8)
plt.ylabel('Total Quantity', fontsize=12)
plt.xlabel('Product Line', fontsize=12)
plt.xticks(rotation='vertical')
plt.title('Totall Quantity by Product Line', fontsize=15)


In [ ]:
unitprice_prod = data.groupby("Product line")["Unit price"].mean().reset_index()
unitprice_prod

In [ ]:
unitprice_prod = data.groupby("Product line")["Unit price"].mean()

sns.pointplot(x=unitprice_prod .index, y=unitprice_prod .values, alpha=0.8)
plt.ylabel('Avg. Unit Price', fontsize=12)
plt.xlabel('Product Line', fontsize=12)
plt.xticks(rotation='vertical')
plt.title('Average Unit Price by Product', fontsize=12)

These unit price are used to calculate COGS and reflect how expensive it is to manufacture these goods, so electronic goods have the cheapest unit price and fashion accessories and sports and travel are the most expensive 

Given each invoice/entry has a different quantity, let's also calculate gross income by one unit of quantity

In [ ]:
data['gross_income_by_unit'] = data['gross income'] / data['Quantity']
data.head()

In [ ]:
sns.boxplot(x="Product line", y="gross_income_by_unit",data=data)
plt.ylabel('Gross Income by Unit', fontsize=12)
plt.xlabel('Product Line', fontsize=12)
plt.xticks(rotation='vertical')
plt.title('Gross Income by Unit', fontsize=15)

In [ ]:
gross_by_unit_prod = data.groupby("Product line")["gross_income_by_unit"].mean()
gross_by_unit_prod

**Customer Analysis**

When do customers shop most frequently?

'Date' is an object so we need to transform it to datetime

In [ ]:
data['weekday'].value_counts().plot(kind="bar")
plt.title("Frequency by Day of the Week", fontsize=15)
plt.show()

Saturday and Tuesday seem to be more popular days than Sunday or Monday

In [ ]:
data['hour'].value_counts().plot(kind="bar", color='pink')
plt.title("Frequency by Hour of the Day", fontsize=15)
plt.show()

Evenings is the most frequent time

In [ ]:
grouped_data = data.groupby(['weekday','hour'])["Invoice ID"].aggregate("count").reset_index()
grouped_data = pd.DataFrame(data=grouped_data)
grouped_data = grouped_data.pivot(index='weekday', columns='hour', values='Invoice ID')

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(grouped_data)
plt.title("Frequency of Day of Week vs. Hour of Day", fontsize=15)
plt.show()

Tuesday evening is the most popular time to shop

In [ ]:
grouped_members = data.groupby('Customer type')["Total"].sum()
sns.barplot(x=grouped_members.index, y=grouped_members.values)
plt.xlabel('Customer type')
plt.ylabel('Total Spent')
plt.title('Spending by Customer Type',fontsize=15)
plt.show()

Members do not spend much more than non-members, there could be an opportunity for a new strategy for a rewards/loyalty program

**Payment Type**

In [ ]:
payment_type = data.groupby("Payment")['Total'].sum()
payment_type.index

In [ ]:
sns.barplot(x=payment_type.index, y=payment_type.values)

Payment type is relatively consistent across the three different groups